In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import opts, dim
#from getLabels import thresholdLabels, getListLabels
import itertools
import numpy as np

hv.extension('bokeh')
hv.output(size=200)

In [ ]:
import pandas
import numpy as np
import re

Labels = set(
    ['Atelectasis', 'Consolidation', 'Edema', 'Pleural_Thickening', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
     'Pneumothorax', 'Cardiomegaly', 'Effusion', 'Hernia',
     'Emphysema', 'Fibrosis', 'No Finding'])

def checkLabel(text, labels):
    findLabels = re.compile(r'([A-Z]\w*(?:\s[A-Z]\w*)?)')
    foundLabel = []
    for possible_labels in set(findLabels.findall(text)):
        if possible_labels in labels:
            foundLabel.append(possible_labels)
    return foundLabel


def getListLabels():

    listLabels = list(Labels)
    tempLabel = listLabels[13]
    listLabels.remove(tempLabel)
    listLabels.append(tempLabel)  # Ordering of the labels are now different (E, Ptho, Ed, C, P.T., Ate, Conso, Emphy,
    # P, Nod, M, Infilt, hernia, No finding)

    return listLabels

def thresholdLabels(threshold=3):
    colName = ['ImageIndex', 'FindingLabels', 'Followup', 'PatientID', 'PatientAge', 'PatientGender', 'ViewPosition',
               'OriginalImage', 'Height', 'OriginalImagePixelSpacing', 'y', 'temp']
    data = pandas.read_csv('../input/data/Data_Entry_2017.csv', names=colName)
    

    findingLabels = data.FindingLabels.tolist()[1:]

    listLabels = getListLabels()

    oneHotLabels = np.zeros((len(findingLabels), 15))
    print(oneHotLabels)
    n = 0
    for i in range(len(findingLabels)):
        returnPathology = checkLabel(findingLabels[i], Labels)

        if len(returnPathology) < threshold and returnPathology[0] != 'No Finding':
            for m in range(len(returnPathology)):
                oneHotLabels[n, listLabels.index(returnPathology[m])] = 1


            n += 1

    thresholdLabels = oneHotLabels[:(n-1)]

    return thresholdLabels


In [ ]:
#Get the one hot encoding of all the possible labels with correlation under certain threshold
THRESHOLD = 3
threshold_Labels = thresholdLabels(THRESHOLD)

In [ ]:

#Change the one hot labels as the dictionary of the combination
table = {}
for j in range(len(threshold_Labels)):
    a = np.sum(threshold_Labels[j])
    b = threshold_Labels[j].tolist()
    indices = [i for i, x in enumerate(b) if x == 1]

    if len(indices) == 1:
        indices.append(indices[0])

    combination = itertools.combinations(indices, 2)

    for subset in combination:
        if subset in table:
            table[subset][2] += 1
        else:
            table[subset] = [subset[0], subset[1], 1]

In [ ]:
#Get dataframe (Since the dictionary in previous step was created using index of tuple, doing some "shameless" hack to remove such tuple as index.)
df = pd.DataFrame.from_dict(table, orient='index')
df.columns = ['source', 'target', 'numbers']

d_temp = df.copy()
d_temp = d_temp.reset_index()

d_temp = d_temp.reset_index()
d_temp_ = d_temp[['source', 'target', 'numbers']].copy()

d_links = d_temp_.copy()

In [ ]:
#Raw version of what we wanted to do!
hv.Chord(d_links)

In [ ]:
#Add labels
listLabels = getListLabels()
d_nodes = d_temp_.copy()
d_nodes['name'] = 'PlaceHolder'
for i in range(len(listLabels)-1):
    d_nodes.loc[d_nodes['source'] == i, 'name'] = listLabels[i]
    
d_nodes.drop(['source', 'target', 'numbers'], 1, inplace=True)

nodes = hv.Dataset(d_nodes, 'index')

In [ ]:
chord = hv.Chord((d_links, nodes)).select(value=(5, None))
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), 
               labels='name', node_color=dim('index').str()))